In [2]:

from config.database import collection, collection_weather, collection_searchTrack, collection_flights, collection_gates
from config.database_UJ import client
from routes.root.root_class import Root_class, Fetching_Mechanism, Source_links_and_api, Root_source_links
from bson import ObjectId
import requests
import datetime as dt
import json
import pickle
from pymongo import UpdateOne

# collection = collection_gates
# collection = client['cirrostrats']['flights']
# collection = collection_searchTrack
# collection.estimated_document_count()

# get all documents. Expensive operation.
# x = [i for i in collection.find({})]

# return the first document
# one = collection.find_one({})
# one

# Delete all documents from the collection
# x = collection.delete_many({})

# match all documents. works like find. will fetch all that matches gjs in `flightNumber` fields
# [i for i in collection_flights.find({'flightNumber':{'$regex': 'GJS'}})]



# user_data = collection.find_one({"email": 'Anonymous'})

In [14]:
from schema.schemas import serialize_document_list
query = "g"    # Search for predertimined items in pipeline containing "ap"
page = 1       # first page
page_size = 50  # items per page
suggestions = []
# print('\n\n\nTriggered /searches/suggestions', query, page, page_size)
collection_merge = [collection, collection_flights] 
for coll in collection_merge:
    pipeline = [
            {"$match": {"count": {"$exists": True}}},        # filter documents that have a count field
            # {"$project": {"_id": 0, "id": {"$toString": "$_id"}, "count": 1}},  # Rename _id to id
            # {"$project": {"_id": {"$toString": "$_id", "count": 1}}},
            {"$match": {"$or": [
                {"flightNumber": {"$regex": query, "$options": "i"}},       # matches flightNumber field in flights collection
                {"name": {"$regex": query, "$options": "i"}}                # matches name field in airport collection
            ]}},
            {"$sort": {"count": -1}},               # sort by popularity - the count field contains popularity rating.
            {"$skip": (page - 1) * page_size},    # the page number itself.
            {"$limit": page_size}                 # items per page
        ]
    coll.aggregate(pipeline)
    suggestions.extend(coll.aggregate(pipeline))
# collection_merge[0].find_one({})
# [i for i in serialize_document_list(suggestions)]  # serialize_document_list(suggestions]  # serialize_document_list(suggestions)


In [ ]:
# adds `count` as 1 to all gjs flights 
with open(r'forMDB.pkl', 'rb') as f:
    forMDB = pickle.load(f)
GJS_scheduled_flights = forMDB['scheduled_flights']
GJS_repo_flights = forMDB['repo_flights']
popularity_raw = forMDB['popularity_raw']
popularity_proportions = forMDB['popularity_proportions']

# 
operations = []
for i in GJS_scheduled_flights:
# for airport, count in popularity_proportions.items():
    flightNumber = "GJS"+str(i)
    operation = UpdateOne(
        {'flightNumber': flightNumber},
        {'$set': {'count': 1}},
        upsert=True
    )
    operations.append(operation)

# Perform bulk write
result = collection_flights.bulk_write(operations)
# popularity_proportions
# collection_flights.find_one()

In [48]:
# documents = collection.find(query)


suggestions = []

query = "g"    # Search for predertimined items in pipeline containing "ap"
page = 1        # 
page_size = 10

collection_merge = [collection, collection_flights] 
for coll in collection_merge:
    pipeline = [
            {"$match": {"count": {"$exists": True}}},
            {"$match": {"$or": [
                {"flightNumber": {"$regex": query, "$options": "i"}},       # matches flightNumber field in flights collection
                {"name": {"$regex": query, "$options": "i"}}                # matches name field in airport collection
            ]}},
            {"$sort": {"count": -1}},               # sort by popularity - the count field contains popularity rating.
            # {"$skip": (page - 1) * page_size},    # the page number itself.
            # {"$limit": page_size}                 # items per page
        ]

    suggestions.extend(coll.aggregate(pipeline))

In [46]:
tot_sug = int(len(suggestions)/20)

x = [suggestions[i:i+20] for i in range(0,len(suggestions),20)]
    
    # new_sug.append(suggestions[i*10:(i+1)*10])
    # break
# slice suggestions into chunks of upto 10 per item in a list making a list of lists
# list(range(0,len(suggestions),10))

# len(x)
# x[1]


In [47]:
# with open(r'test_popular_suggestions.pkl', 'wb') as f:
    # pickle.dump(x, f)

In [ ]:
# New UJ collection

# from config.database_UJ import collection, collection_weather, collection_searchTrack
from pymongo import UpdateOne
from bson import ObjectId
from config.database_UJ import client
client.list_database_names()

db = client['cirrostrats']      # create a db
collection = db['']        # create a collection

# all_db = db.list_collection_names()   # get all collections of the db
# db = client.cirrostrats               # get a particular db
# collection = db['flights']               # get a particular collection

# collection.estimated_document_count() # get the number of documents in the collection.

# docs = [{'flightNumber':i} for i in flightNumbers]        # add flight numbers to the collection- fast operation
# collection.insert_many(docs)

# # This will insert flightnumbers in bulk in the collection if it doesnt exist - slow operation since it checks if the flightnumber already exist.
# update_operations = []
# for i in flightNumbers:
#     update_operations.append(
#         UpdateOne({'flightNumber': i},
#                   {'$set': {'flightNumber': i}},
#                   upsert=True
#                   )
#     )
# result = collection_flights.bulk_write(update_operations)



In [3]:
import requests
xx = requests.get("http://3.15.228.76:8000/flights")

In [105]:

repo_flights = []
scheduled_flights = []
for i in xx.json()['flightNumbers']:
    # first three digits are GJS, after those digits are flight number
    # if len(i) == 7 and i[:3] == 'UAL' and i[3:].isnumeric():
    if i[:3] == 'UAL' and i[3:].isnumeric():
        # if i[3] == '4':
        scheduled_flights.append(int(i[3:]))
        # elif i[3] == '3':
            # repo_flights.append(int(i[3:]))


In [112]:
len(scheduled_flights)
# scheduled_flights_RPA = scheduled_flights

2340

In [133]:
jmsFlightNumbers = xx.json()['flightNumbers']
# Saving these list of flight numbers into the collection flightNumbers

# collection_flightNumbers.insert_many(jmsFlightNumbers)



In [134]:
import pickle
with open('JMS_flight_numbers.pkl', 'wb') as f:
    pickle.dump(jmsFlightNumbers, f)

In [ ]:

from routes.root.weather_parse import Weather_parse
from routes.root.dep_des import Pull_flight_info

tots = {}
flt_info = Pull_flight_info()
for i in scheduled_flights:
    rets = flt_info.fs_dep_arr_timezone_pull(i)
    tots[rets['origin_fs']] = tots.get(rets['origin_fs'],0) + 1
    tots[rets['destination_fs']] = tots.get(rets['destination_fs'],0) + 1
    
# Sorting the dictionary by values
tots = {k: v for k, v in sorted(tots.items(), key=lambda item: item[1], reverse=True)}
tots

In [ ]:
new_tots = {}
for a,b in tots.items():
    sum_of_values = sum(tots.values())
    new_tots[a] = int(b/sum_of_values*100)+1

# list total number of values or keys in the dictionary
tots

In [95]:
import pickle

forMDB = {'scheduled_flights': scheduled_flights, 'repo_flights': repo_flights, 'popularity_raw': tots, 'popularity_proportions': new_tots}

with open(r'forMDB.pkl', 'wb') as f:
    pickle.dump(forMDB, f)

In [ ]:
# from config.database_UJ import collection, collection_weather, collection_searchTrack
from config.database_UJ import client

# list all db
# client.list_database_names()

# Make a new db
# db = client.['db_name']

# Make a new collection
# collection = db['collection_name']

# list all collections
# all_db = db.list_collection_names()

client['']

In [1]:
import requests


x = requests.get("http://3.15.228.76:8000/flights")

In [5]:
import json
with open('testJmsFlights.json', 'w') as file:
    # Write data to file
    json.dump(x.json(), file)

In [ ]:
for data in x:

    update_query = {
        "$setOnInsert": {"email": data['email']},  # Only set email on document creation
        "$inc": {f"searches.{data['searchTerm']}": 1},  # Increment count for this search term
        "$set": {"lastUpdated": data['timestamp']}  # Update timestamp
    }
    # This single operation will:
    # 1. Create document if email doesn't exist
    # 2. Create searchTerm with count 1 if it doesn't exist
    # 3. Increment count if searchTerm exists
    result = collection_searchTrack.update_one(
        {"email": data['email']},
        update_query,
        upsert=True
    )

    print({"status": "success", "matched_count": result.matched_count, "modified_count": result.modified_count})

In [ ]:
# searches
y = [i for i in collection_searchTrack.find({})]
y

In [ ]:
import requests

# extrac the data from the response - all flight numbers
db = requests.get("http://3.15.228.76:8000/db")

response = db.json()
result = response['db'] 

db = [i for i in result]
print("total flights", len(db))


total flights 35205


In [25]:
import pickle
with open('flight_data.pkl', 'rb') as f:
    xx = pickle.load(f)

# with open('flight_data.pkl', 'wb') as f:
#     pickle.dump(x, f)

In [ ]:
gjs_flight_numbers = [i for i in x if i[:3] == 'GJS']

def gjs_returns(flightNumber):
    return requests.get(f"http://3.15.228.76:8000/flightData/{flightNumber}")


gjs_flight_data = [gjs_returns(i).json() for i in gjs_flight_numbers]

In [16]:
registration = []
for i in a:
    registration.append(i['flightData']['registration'])

print(len(registration),len(set(registration)))

169 38


In [17]:
set(registration)

{'N503GJ',
 'N504GJ',
 'N506GJ',
 'N508GJ',
 'N511GJ',
 'N520GJ',
 'N521GJ',
 'N522GJ',
 'N523GJ',
 'N524GJ',
 'N526GJ',
 'N534GJ',
 'N535GJ',
 'N536GJ',
 'N537GJ',
 'N538GJ',
 'N539GJ',
 'N540GJ',
 'N541GJ',
 'N543GJ',
 'N544GJ',
 'N546GJ',
 'N548GJ',
 'N549GJ',
 'N551GJ',
 'N552GJ',
 'N554GJ',
 'N556GJ',
 'N557GJ',
 'N559GJ',
 'N561GJ',
 'N563GJ',
 'N564GJ',
 'N566GJ',
 'N569GJ',
 'N578GJ',
 'N579GJ',
 None}